### tf.keras Practice!

- CIFAR10 이미지 데이터셋을 분류하는 문제를 풀어봅니다!

In [1]:
!nvidia-smi

Wed Nov 30 05:57:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.9.2'

In [3]:
# DNN
from tensorflow import keras
import tensorflow as tf
# datasets
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras import Sequential

In [4]:
# load CIFAR10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
## RGB (3-channel)
# 32 x 32 - R -> pixel matrix
# 32 x 32 - G
# 32 x 32 - B

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


### Model Training

1. model architecture

- Hidden Layer (뒤로 갈수록 hidden node수가 작아짐)

- Activation function : 'sigmoid' / 'relu'



2. loss optimization


- optimizer : 'sgd', 'momentum', 'adam'

- batch_size : 1 -> 8-> 32 -> 64 -> 128 -> 256 -> 512 -> 2048 -> ...

- epochs : 10 -> 30 -> 100 -> 10000 (overfitting) 

In [5]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Flatten(),  # 32x32x3 ---> 3072  (input layer의 node 수)
    Dense(units=1024, activation='relu'),
    Dense(units=512, activation='relu'),
    Dense(units=256, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(units=10, activation='softmax') # output layer
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 1024)              3146752   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 32)                2

In [6]:
y_train # 0 ~ 9

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [7]:
# compile
from tensorflow.keras.optimizers import SGD, Adam

lr = 1e-4
epochs = 200
batch_size = 128
# 모든 데이터가 학습에 다 사용되면 1 epoch.
# epochs : 전체 데이터를 몇번 반복 학습을 수행할건지.
# iterations : 실제로 weight update를 한 횟수.

loss_fn = 'sparse_categorical_crossentropy' # target vector가 정수인 경우.
#optimizer = SGD(learning_rate=lr) # learning rate 조절을 위해서 보통 함수로 구현.
#optimizer = SGD(learning_rate=lr, momentum=0.9) # Momentum optimizer
optimizer = Adam(learning_rate=lr) # Adam optimizer

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

In [8]:
# fit
model.fit(x=X_train,
          y=y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          validation_data=(X_test, y_test))

Epoch 1/200
391/391 - 3s - loss: 3.2410 - accuracy: 0.1700 - val_loss: 2.1638 - val_accuracy: 0.2188 - 3s/epoch - 9ms/step
Epoch 2/200
391/391 - 3s - loss: 2.1393 - accuracy: 0.2302 - val_loss: 2.0876 - val_accuracy: 0.2579 - 3s/epoch - 6ms/step
Epoch 3/200
391/391 - 1s - loss: 2.0454 - accuracy: 0.2750 - val_loss: 2.0097 - val_accuracy: 0.2846 - 1s/epoch - 4ms/step
Epoch 4/200
391/391 - 2s - loss: 1.9606 - accuracy: 0.3122 - val_loss: 1.9134 - val_accuracy: 0.3231 - 2s/epoch - 4ms/step
Epoch 5/200
391/391 - 1s - loss: 1.8933 - accuracy: 0.3419 - val_loss: 1.8593 - val_accuracy: 0.3478 - 1s/epoch - 4ms/step
Epoch 6/200
391/391 - 1s - loss: 1.8241 - accuracy: 0.3613 - val_loss: 1.8148 - val_accuracy: 0.3705 - 1s/epoch - 4ms/step
Epoch 7/200
391/391 - 2s - loss: 1.7677 - accuracy: 0.3818 - val_loss: 1.7899 - val_accuracy: 0.3746 - 2s/epoch - 4ms/step
Epoch 8/200
391/391 - 1s - loss: 1.7259 - accuracy: 0.3951 - val_loss: 1.7926 - val_accuracy: 0.3844 - 1s/epoch - 4ms/step
Epoch 9/200
391/

In [9]:
## 어떤 Hyper-parameter setting에서 최고의 성능이 나왔나요?
# evaluate
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("Loss : %.4f, Test Accuracy : %.4f" % (loss, acc))

79/79 [==============================] - 0s 3ms/step - loss: 5.3294 - accuracy: 0.5212
Loss : 5.3294, Test Accuracy : 0.5212
